In [1]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import lightgbm as lgb
import talib

import src
from src.ml_utils import (
    fetch_ohlcv, 
    visualize_result, 
    normalize_position, 
    calc_position_cv,
    get_feature_columns,
    get_symbols,
    unbiased_rank,
    ewm_finite,
)
cloudpickle.register_pickle_by_value(src) # for model portability

In [29]:
# symbols = 'BTC'.split(',')
symbols = 'ATOM'.split(',')
# symbols = os.getenv('ALPHAPOOL_SYMBOLS').split(',') # 売買代金が多く、古いもの
df = fetch_ohlcv(symbols=symbols, with_target=False, interval_sec=300, price_type=None)
df.to_pickle('/tmp/df.pkl')
display(df)

,,op,hi,lo,cl,volume,execution_start_at
timestamp,symbol,,,,,,
2019-11-12 00:30:00+00:00,ATOM,4.0580,4.2375,4.0530,4.2375,11876.011855,2019-11-12 00:35:00+00:00
2019-11-12 00:35:00+00:00,ATOM,4.2405,4.3070,4.2405,4.3070,4017.311465,2019-11-12 00:40:00+00:00
2019-11-12 00:40:00+00:00,ATOM,4.3070,4.3180,4.3070,4.3180,0.000000,2019-11-12 00:45:00+00:00
2019-11-12 00:45:00+00:00,ATOM,4.3180,4.3285,4.3180,4.3270,0.000000,2019-11-12 00:50:00+00:00
2019-11-12 00:50:00+00:00,ATOM,4.3270,4.3360,4.3270,4.3345,2202.970665,2019-11-12 00:55:00+00:00
...,...,...,...,...,...,...,...
2022-08-30 04:05:00+00:00,ATOM,11.6000,11.6160,11.5875,11.6070,18442.659740,2022-08-30 04:10:00+00:00
2022-08-30 04:10:00+00:00,ATOM,11.6070,11.6355,11.6070,11.6200,149026.916845,2022-08-30 04:15:00+00:00
2022-08-30 04:15:00+00:00,ATOM,11.6200,11.6710,11.6105,11.6635,206299.359880,2022-08-30 04:20:00+00:00


In [40]:
df = pd.read_pickle('/tmp/df.pkl')

import numba
@numba.njit
def calc_force_entry_price(entry_price=None, lo=None, pips=None):
    y = entry_price.copy()
    y[:] = np.nan
    for i in range(entry_price.size):
        for j in range(i + 1, entry_price.size):
            if round(lo[j] / pips[j]) < round(entry_price[j - 1] / pips[j]):
                y[i] = entry_price[j - 1]
                break
    return y


def estimate_pips(x, window=None, reverse=False):
    if reverse:
        return estimate_pips(x[::-1], window=window)[::-1]
    scale = 10 ** 8
    x = np.round(x * scale).astype(np.long)
    if x.ndim != 1:
        x = np.gcd.reduce(x, axis=1)
    x2 = x.copy()
    for i in range(1, window):
        x2[i:] = np.gcd(x2[i:], x[:-i])
    return x2 * 1.0 / scale

df['pips'] = estimate_pips(df[['op', 'hi', 'lo', 'cl']], window=1000)

df['buy_price'] = df['cl'] - df['pips'] * 1
df['sell_price'] = df['cl'] + df['pips'] * 1
# df['buy_price'] = df['cl'] - df['cl'].rolling(12, 1).std() * 0.5
# df['sell_price'] = df['cl'] + df['cl'].rolling(12, 1).std() * 0.5
# df['buy_price'] = np.floor(df['buy_price'] / df['pips']) * df['pips']
# df['sell_price'] = np.ceil(df['sell_price'] / df['pips']) * df['pips']

df['buy_fep'] = calc_force_entry_price(
    entry_price=df['buy_price'].values,
    lo=df['lo'].values,
    pips=df['pips'].values
)

df['sell_fep'] = -calc_force_entry_price(
    entry_price=-df['sell_price'].values,
    lo=-df['hi'].values,
    pips=df['pips'].values
)

df['signal'] = -np.sign(df['cl'] - df['cl'].shift(1))
df['ret'] = df['cl'].shift(-1) / df['cl'] - 1

df['buy_fep_prev'] = df['buy_fep'].shift(1)
df['sell_fep_prev'] = df['sell_fep'].shift(1)

df = df.dropna()

print('corr', df['ret'].corr(df['signal']))

df['buy_cost'] = df['buy_fep'] / df['cl'] - 1
df['sell_cost'] = -(df['sell_fep'] / df['cl'] - 1)
df['buy_cost_early'] = df['buy_fep_prev'] / df['cl'] - 1
df['sell_cost_early'] = -(df['sell_fep_prev'] / df['cl'] - 1)
display(df.describe())

print('up')
display(df.loc[df['ret'] > 0].describe())
print('down')
display(df.loc[df['ret'] < 0].describe())

/tmp/ipykernel_29/2856714753.py:20: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.round(x * scale).astype(np.long)


corr -0.007264933295255931


,op,hi,lo,cl,volume,pips,buy_price,sell_price,buy_fep,sell_fep,signal,ret,buy_fep_prev,sell_fep_prev,buy_cost,sell_cost,buy_cost_early,sell_cost_early
count,294364.000000,294364.000000,294364.000000,294364.000000,2.943640e+05,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000,294364.000000
mean,13.882180,13.924415,13.839652,13.882208,1.571289e+05,0.000499,13.881709,13.882707,13.886649,13.877993,-0.002952,0.000014,13.886624,13.877968,0.000470,0.000433,0.000474,0.000429
std,11.086261,11.122964,11.049698,11.086254,4.035944e+05,0.000015,11.086254,11.086255,11.087847,11.084608,0.982414,0.004657,11.087860,11.084621,0.002110,0.002042,0.004410,0.004408
min,1.110500,1.122500,1.062000,1.103500,0.000000e+00,0.000250,1.103000,1.104000,1.120500,1.104000,-1.000000,-0.250196,1.120500,1.104000,-0.000750,-0.000737,-0.153305,-0.333732
25%,4.524500,4.533500,4.516000,4.525000,3.069360e+03,0.000500,4.524500,4.525500,4.527000,4.522375,-1.000000,-0.001902,4.527000,4.522000,-0.000095,-0.000097,-0.001060,-0.001109
50%,9.718500,9.762500,9.680750,9.718500,3.834044e+04,0.000500,9.718000,9.719000,9.722250,9.715000,-0.000000,0.000000,9.722000,9.715000,-0.000037,-0.000038,0.000049,0.000025
75%,22.717000,22.800500,22.645000,22.717500,1.506920e+05,0.000500,22.717000,22.718000,22.724000,22.711125,1.000000,0.001909,22.724000,22.711125,-0.000017,-0.000017,0.002058,0.002018
max,44.444000,44.802500,44.228000,44.444000,1.799882e+07,0.000500,44.443500,44.444500,44.443500,44.444500,1.000000,0.180636,44.443500,44.444500,0.163570,0.111544,0.333632,0.152693


up


,op,hi,lo,cl,volume,pips,buy_price,sell_price,buy_fep,sell_fep,signal,ret,buy_fep_prev,sell_fep_prev,buy_cost,sell_cost,buy_cost_early,sell_cost_early
count,142485.000000,142485.000000,142485.000000,142485.000000,1.424850e+05,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000,142485.000000
mean,14.118189,14.161166,14.074093,14.117396,1.612855e+05,0.000499,14.116896,14.117895,14.126928,14.117881,0.000049,0.003039,14.123311,14.114383,0.000992,-0.000068,0.000638,0.000277
std,11.051289,11.087743,11.014691,11.050863,4.187315e+05,0.000014,11.050862,11.050863,11.054113,11.050874,0.987396,0.003643,11.052737,11.049598,0.002872,0.000176,0.004664,0.004490
min,1.110500,1.122500,1.062000,1.103500,0.000000e+00,0.000250,1.103000,1.104000,1.284000,1.104000,-1.000000,0.000012,1.120500,1.104000,-0.000500,-0.000737,-0.147304,-0.333732
25%,4.683000,4.692000,4.672000,4.682500,3.949920e+03,0.000500,4.682000,4.683000,4.689500,4.683000,-1.000000,0.000871,4.687000,4.680000,-0.000061,-0.000106,-0.001081,-0.001188
50%,10.266000,10.293500,10.232500,10.262500,4.152668e+04,0.000500,10.262000,10.263000,10.269500,10.263000,0.000000,0.001997,10.268000,10.262000,-0.000022,-0.000049,0.000254,-0.000023
75%,22.962500,23.027500,22.878000,22.956000,1.550940e+05,0.000500,22.955500,22.956500,22.975000,22.956500,1.000000,0.003951,22.974000,22.957500,0.000235,-0.000022,0.002329,0.001853
max,44.242500,44.544500,44.200000,44.352500,1.799882e+07,0.000500,44.352000,44.353000,44.352000,44.353000,1.000000,0.180636,44.242000,44.243000,0.163570,0.024881,0.333632,0.147219


down


,op,hi,lo,cl,volume,pips,buy_price,sell_price,buy_fep,sell_fep,signal,ret,buy_fep_prev,sell_fep_prev,buy_cost,sell_cost,buy_cost_early,sell_cost_early
count,141618.000000,141618.000000,141618.000000,141618.000000,1.416180e+05,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000,141618.000000
mean,14.351654,14.395769,14.308038,14.352494,1.634689e+05,0.000499,14.351994,14.352993,14.352006,14.343361,-0.004357,-0.003028,14.355561,14.346872,-0.000066,0.000927,0.000297,0.000565
std,11.144655,11.181481,11.108238,11.145033,4.001744e+05,0.000014,11.145032,11.145033,11.145021,11.141876,0.987499,0.003643,11.146355,11.143114,0.000164,0.002786,0.004241,0.004420
min,1.163500,1.163500,1.062500,1.121000,0.000000e+00,0.000250,1.120500,1.121500,1.120500,1.121500,-1.000000,-0.250196,1.163000,1.121500,-0.000750,-0.000446,-0.153305,-0.176825
25%,4.746625,4.755000,4.736500,4.747000,4.200944e+03,0.000500,4.746500,4.747500,4.746500,4.740500,-1.000000,-0.003973,4.748000,4.743000,-0.000104,-0.000062,-0.001156,-0.001156
50%,10.480000,10.513500,10.444500,10.482250,4.406991e+04,0.000500,10.481750,10.482750,10.481750,10.473000,-0.000000,-0.002009,10.482500,10.474250,-0.000047,-0.000023,-0.000023,0.000223
75%,23.227375,23.303375,23.158000,23.226000,1.589921e+05,0.000500,23.225500,23.226500,23.225500,23.211875,1.000000,-0.000868,23.237750,23.220000,-0.000021,-0.000012,0.001846,0.002247
max,44.444000,44.802500,44.228000,44.444000,1.699900e+07,0.000500,44.443500,44.444500,44.443500,44.444500,1.000000,-0.000012,44.443500,44.444500,0.021177,0.111544,0.176769,0.152693
